# Concatenate multiple Excel files into one dataFrame with `pandas`

We have a directory with a bunch of Excel files. How can we easily combine them all without weird lists or overly complicated `glob` calls? Let's try to use as much plain Python and `pandas` as possible 

In [147]:
import pandas as pd
import glob, os

In [148]:
df = pd.concat(map(pd.read_excel, ['UCSD Dept of Music 2013-2014 Concert Archive.xlsx', 'UCSD Dept of Music 2014-2015 Concert Archive.xlsx','UCSD Dept of Music 2015-2016 Concert Archive.xlsx', 'UCSD Dept of Music 2016-2017 Concert Archive.xlsx', 'UCSD Dept of Music 2018-2019 Concert Archive.xlsx']), sort=False)

In [149]:
df.describe()

,Event Date,Event Title,TITLE,COMPOSER_CREATOR,LOCATION,DATE_CREATED,COMMENTS,Filename,Event_Code,TIME,ca_contrib::Instrument,ca_contrib::Name,Department Resources::Recording Engineer,Primary_FORMAT,Department Resources::Classification,Department Resources::Contact,Academic Quarter,Event Year,Department Resources::Quarter,Department Resources::Year
count,3857,3854,3822,3286,3857,982,203,3857,3858,3670,9080,9715,3137,3857,3854,3854,2323,2319,1535,1535
unique,569,506,3377,1583,18,293,173,3855,735,1528,428,1641,30,2,18,182,3,3,3,2
top,2014-05-29 00:00:00,Chamber Ensembles,Encore,Ludwig van Beethoven,CPMC Concert Hall,2014,Kronomorfic,20140414-CamLu7-01-Brahms.wav,20190531-UGForum3,00:04:17,Piano,"Matsuno, Michael",Ulysses Nieto,96KHz/24bit stereo wav,Class,Diane Salisbury,Spring,2016-17,Spring,2018-19
freq,34,161,15,47,2023,56,8,2,26,12,1095,157,483,3667,1326,264,1006,811,632,837
first,2013-09-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2019-06-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10194 entries, 0 to 2157
Data columns (total 20 columns):
Event Date                                  3857 non-null datetime64[ns]
Event Title                                 3854 non-null object
TITLE                                       3822 non-null object
COMPOSER_CREATOR                            3286 non-null object
LOCATION                                    3857 non-null object
DATE_CREATED                                982 non-null object
COMMENTS                                    203 non-null object
Filename                                    3857 non-null object
Event_Code                                  3858 non-null object
TIME                                        3670 non-null object
ca_contrib::Instrument                      9080 non-null object
ca_contrib::Name                            9715 non-null object
Department Resources::Recording Engineer    3137 non-null object
Primary_FORMAT                           

Dates are always a problem, even in `pandas`. The short version is that no matter what kind of functions we run on a `datetime` type column, it will remain unchanged at the data level (we can make it display differently, but that's not important to our data needs). So, we should change the data from `datetime` to a `string` type:

In [151]:
df['Event Date'] = df['Event Date'].dt.strftime('%Y-%m-%d')

Our "NA" values (by default "NaN") will get in the way, and Excel/Refine expect nulls, so let's fill that in

In [152]:
df = df.fillna('')

Unfortaunately, this didn't touch our date "NaT" values, since those are now strings! We couldn't replace these initially because then the string conversion would error out. So it's ugly, but we'll do a regular expression replacement on that column

In [154]:
df.replace({'Event Date': r'NaT'}, {'Event Date': ''}, regex=True, inplace=True)

In [156]:
df[0:19]

,Event Date,Event Title,TITLE,COMPOSER_CREATOR,LOCATION,DATE_CREATED,COMMENTS,Filename,Event_Code,TIME,ca_contrib::Instrument,ca_contrib::Name,Department Resources::Recording Engineer,Primary_FORMAT,Department Resources::Classification,Department Resources::Contact,Academic Quarter,Event Year,Department Resources::Quarter,Department Resources::Year
0,2013-09-23,Grad Welcome Concert,Improvisation,Bonnie Lander,CPMC Concert Hall,,,20130923-GradWelcome-01-Imp.wav,20130923-GradWelcome,00:06:44,Vocalist,"Lander, Bonnie",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
1,2013-09-23,Grad Welcome Concert,Lied/Lied_01,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-02-Lie.wav,20130923-GradWelcome,00:00:26,Violin,"MacAdam-Somer, Batya",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
2,2013-09-23,Grad Welcome Concert,Psappha,Iannis Xenakis,CPMC Concert Hall,,,20130923-GradWelcome-03-Psa.wav,20130923-GradWelcome,00:13:26,Percussion,"Hepfer, Jon",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
3,2013-09-23,Grad Welcome Concert,Lied/Lied_02,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-04-Lie.wav,20130923-GradWelcome,00:00:20,Violin,"MacAdam-Somer, Batya",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
4,2013-09-23,Grad Welcome Concert,Piano Sonata No. 7,Viktor Ullmann,CPMC Concert Hall,,,20130923-GradWelcome-05-No7.wav,20130923-GradWelcome,00:03:52,Piano,"Lee, Siu Hei",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
5,2013-09-23,Grad Welcome Concert,Lied/Lied_03,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-06-Lie.wav,20130923-GradWelcome,00:00:45,Violin,"MacAdam-Somer, Batya",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
6,2013-09-23,Grad Welcome Concert,Invocaciones,Roberto Sierra,CPMC Concert Hall,,,20130923-GradWelcome-07-Inv.wav,20130923-GradWelcome,00:06:21,Soprano,"DuMouchelle, Tiffany",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
7,,,,,,,,,,,Percussion,"Solook, Stephen",,,,,,,,
8,2013-09-23,Grad Welcome Concert,Lied/Lied_04,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-08-Lie.wav,20130923-GradWelcome,00:01:23,Violin,"MacAdam-Somer, Batya",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,
9,2013-09-23,Grad Welcome Concert,Ultima,Philippe Manoury,CPMC Concert Hall,,,20130923-GradWelcome-09-Ult.wav,20130923-GradWelcome,00:12:46,Clarinet,"Miller, Curt",Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,


Inspecting the data, we see that there are two columns that represent the academic quarter, which probably happened because different sheets had unqiue column names. Let's merge those into one 'Quarter' column

In [157]:
df = df.assign(Quarter = df['Academic Quarter'].astype(str) + df['Department Resources::Quarter'].astype(str))

We'll do the same for year, which had the same issue as quarter

In [158]:
df = df.assign(Year = df['Event Year'].astype(str) + df['Department Resources::Year'].astype(str))

In [159]:
df[0:29]

,Event Date,Event Title,TITLE,COMPOSER_CREATOR,LOCATION,DATE_CREATED,COMMENTS,Filename,Event_Code,TIME,...,Department Resources::Recording Engineer,Primary_FORMAT,Department Resources::Classification,Department Resources::Contact,Academic Quarter,Event Year,Department Resources::Quarter,Department Resources::Year,Quarter,Year
0,2013-09-23,Grad Welcome Concert,Improvisation,Bonnie Lander,CPMC Concert Hall,,,20130923-GradWelcome-01-Imp.wav,20130923-GradWelcome,00:06:44,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
1,2013-09-23,Grad Welcome Concert,Lied/Lied_01,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-02-Lie.wav,20130923-GradWelcome,00:00:26,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
2,2013-09-23,Grad Welcome Concert,Psappha,Iannis Xenakis,CPMC Concert Hall,,,20130923-GradWelcome-03-Psa.wav,20130923-GradWelcome,00:13:26,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
3,2013-09-23,Grad Welcome Concert,Lied/Lied_02,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-04-Lie.wav,20130923-GradWelcome,00:00:20,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
4,2013-09-23,Grad Welcome Concert,Piano Sonata No. 7,Viktor Ullmann,CPMC Concert Hall,,,20130923-GradWelcome-05-No7.wav,20130923-GradWelcome,00:03:52,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
5,2013-09-23,Grad Welcome Concert,Lied/Lied_03,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-06-Lie.wav,20130923-GradWelcome,00:00:45,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
6,2013-09-23,Grad Welcome Concert,Invocaciones,Roberto Sierra,CPMC Concert Hall,,,20130923-GradWelcome-07-Inv.wav,20130923-GradWelcome,00:06:21,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
7,,,,,,,,,,,...,,,,,,,,,,
8,2013-09-23,Grad Welcome Concert,Lied/Lied_04,Nicholas Deyoe,CPMC Concert Hall,,,20130923-GradWelcome-08-Lie.wav,20130923-GradWelcome,00:01:23,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14
9,2013-09-23,Grad Welcome Concert,Ultima,Philippe Manoury,CPMC Concert Hall,,,20130923-GradWelcome-09-Ult.wav,20130923-GradWelcome,00:12:46,...,Daniel Ross,96KHz/24bit stereo wav,Forum,Rachel Beetz,Fall,2013-14,,,Fall,2013-14


So this data is pretty much ready for export (skip to the bottom of the notebook for that).  

But what if we don't want to plug in the filenames like we did at the beginning? Let's make a quick dataFrame using that method 

In [71]:
df1 = pd.concat(map(pd.read_excel, glob.glob('*.xlsx')), sort=False)

In [21]:
df1[0:29]

,Event Date,Event Title,TITLE,COMPOSER_CREATOR,LOCATION,DATE_CREATED,COMMENTS,Filename,Event_Code,TIME,ca_contrib::Instrument,ca_contrib::Name,Department Resources::Recording Engineer,Primary_FORMAT,Department Resources::Classification,Department Resources::Contact,Academic Quarter,Event Year,Department Resources::Quarter,Department Resources::Year
0,2016-09-27,Computer Music Focus: Andrew Brown,Lecture,Andrew Brown,CPMC Recital Hall,NaN,NaN,20160927-CMF1Brown-01-lecture.wav,20160927-CMF1Brown,01:03:28,Lecturer\n,"Brown, Andrew",Kevin Di Bella,96KHz/24bit stereo wav,Guest,Shlomo Dubnov / Andrew Brown,Fall,2016-17,NaN,NaN
1,2016-10-05,"WEDS@7 Aleck Karis, piano",Etudes. book 1,Claude Debussy,CPMC Concert Hall,1915,NaN,20161005-WEDS7Karis-01-Debussy.wav,20161005-WEDS7Karis,00:24:17,Piano,"Karis, Aleck",James (Forest) Reid,96KHz/24bit stereo wav,WEDS7,Aleck Karis,Fall,2016-17,NaN,NaN
2,2016-10-05,"WEDS@7 Aleck Karis, piano",Exegesis sobre La Galatea,Victor Ibarra,CPMC Concert Hall,2016,NaN,20161005-WEDS7Karis-02-Ibarra.wav,20161005-WEDS7Karis,00:12:18,Piano,"Karis, Aleck",James (Forest) Reid,96KHz/24bit stereo wav,WEDS7,Aleck Karis,Fall,2016-17,NaN,NaN
3,2016-10-05,"WEDS@7 Aleck Karis, piano",The moon is following us,Lei Liang,CPMC Concert Hall,2015,NaN,20161005-WEDS7Karis-03-Liang.wav,20161005-WEDS7Karis,00:12:17,Piano,"Karis, Aleck",James (Forest) Reid,96KHz/24bit stereo wav,WEDS7,Aleck Karis,Fall,2016-17,NaN,NaN
4,2016-10-05,"WEDS@7 Aleck Karis, piano",Harrison’s Clocks,Harrison Birtwistle,CPMC Concert Hall,1998,NaN,20161005-WEDS7Karis-05-Birtwistle.wav,20161005-WEDS7Karis,00:30:08,Piano,"Karis, Aleck",James (Forest) Reid,96KHz/24bit stereo wav,WEDS7,Aleck Karis,Fall,2016-17,NaN,NaN
5,2016-10-05,"WEDS@7 Aleck Karis, piano",Birtwistle Intro,NaN,CPMC Concert Hall,NaN,NaN,20161005-WEDS7Karis-04-Intro.wav,20161005-WEDS7Karis,00:05:16,Piano,"Karis, Aleck",James (Forest) Reid,96KHz/24bit stereo wav,WEDS7,Aleck Karis,Fall,2016-17,NaN,NaN
6,2016-10-05,"WEDS@7 Aleck Karis, piano",Tango,Igor Stravinsky,CPMC Concert Hall,1940,NaN,20161005-WEDS7Karis-06-Stravinsky.wav,20161005-WEDS7Karis,00:04:04,Piano,"Karis, Aleck",James (Forest) Reid,96KHz/24bit stereo wav,WEDS7,Aleck Karis,Fall,2016-17,NaN,NaN
7,2016-10-07,The Voice Machine,How to Survive a 100-Hour Workweek,Caroline Miller,CPMC Experimental Theatre,NaN,NaN,20161007-StanfordOpera-01-HowTo.wav,20161007-StanfordOpera,00:17:26,Soprano,"Young, Hillary Jean",Daniel McFarland,96KHz/24bit stereo wav,Class,Susan Narucki,Fall,2016-17,NaN,NaN
8,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soprano,"Wiest, Kirsten",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mezzo-Soprano,"Cutright, Ashley",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


So let's go back to our original dataFrame, and write it out to one big merged Excel sheet

In [160]:
writer = pd.ExcelWriter('/home/zelgius/Documents/dmca_merged.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Item_description')
writer.save()

Once this Excel sheet is exported, we can import it into OpenRefine and do the more complex data wrangling!